In [1]:
import os
path = os.getcwd()
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("github_csv").getOrCreate()
from pyspark.sql.functions import max, desc

In [2]:
spark = SparkSession.builder.appName("github_csv").getOrCreate()

22/07/01 15:25:04 WARN Utils: Your hostname, DESKTOP-CHAV9A4 resolves to a loopback address: 127.0.1.1; using 172.20.116.69 instead (on interface eth0)
22/07/01 15:25:04 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/07/01 15:25:05 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
myRange = spark.range(1000).toDF("number")
myRange
divisBy2 = myRange.where("number % 2 = 0")
divisBy2.head()

Row(number=0)

In [13]:
flightData2015 = spark\
  .read\
  .option("inferSchema", "true")\
  .option("header", "true")\
  .csv("../data/flight-data/csv/2015-summary.csv")
flightData2015.createOrReplaceTempView("flight_data_2015")

In [14]:
sqlWay = spark.sql("""
SELECT DEST_COUNTRY_NAME, count(1)
FROM flight_data_2015
GROUP BY DEST_COUNTRY_NAME
""")

dataFrameWay = flightData2015\
  .groupBy("DEST_COUNTRY_NAME")\
  .count()

sqlWay.explain()
dataFrameWay.explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- HashAggregate(keys=[DEST_COUNTRY_NAME#23], functions=[count(1)])
   +- Exchange hashpartitioning(DEST_COUNTRY_NAME#23, 200), ENSURE_REQUIREMENTS, [id=#49]
      +- HashAggregate(keys=[DEST_COUNTRY_NAME#23], functions=[partial_count(1)])
         +- FileScan csv [DEST_COUNTRY_NAME#23] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/home/duonghung/Self-study-Spark-The-Definitive-Guide/data/flight..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string>


== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- HashAggregate(keys=[DEST_COUNTRY_NAME#23], functions=[count(1)])
   +- Exchange hashpartitioning(DEST_COUNTRY_NAME#23, 200), ENSURE_REQUIREMENTS, [id=#62]
      +- HashAggregate(keys=[DEST_COUNTRY_NAME#23], functions=[partial_count(1)])
         +- FileScan csv [DEST_COUNTRY_NAME#23] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFi

In [15]:
flightData2015.select(max("count")).take(1)

[Row(max(count)=370002)]

In [16]:
maxSql = spark.sql("""
SELECT DEST_COUNTRY_NAME, sum(count) as destination_total
FROM flight_data_2015
GROUP BY DEST_COUNTRY_NAME
ORDER BY sum(count) DESC
LIMIT 5
""")
maxSql.show()

+-----------------+-----------------+
|DEST_COUNTRY_NAME|destination_total|
+-----------------+-----------------+
|    United States|           411352|
|           Canada|             8399|
|           Mexico|             7140|
|   United Kingdom|             2025|
|            Japan|             1548|
+-----------------+-----------------+



In [17]:
flightData2015\
  .groupBy("DEST_COUNTRY_NAME")\
  .sum("count")\
  .withColumnRenamed("sum(count)", "destination_total")\
  .sort(desc("destination_total"))\
  .limit(5)\
  .show()

+-----------------+-----------------+
|DEST_COUNTRY_NAME|destination_total|
+-----------------+-----------------+
|    United States|           411352|
|           Canada|             8399|
|           Mexico|             7140|
|   United Kingdom|             2025|
|            Japan|             1548|
+-----------------+-----------------+



In [18]:
flightData2015\
  .groupBy("DEST_COUNTRY_NAME")\
  .sum("count")\
  .withColumnRenamed("sum(count)", "destination_total")\
  .sort(desc("destination_total"))\
  .limit(5)\
  .explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- TakeOrderedAndProject(limit=5, orderBy=[destination_total#105L DESC NULLS LAST], output=[DEST_COUNTRY_NAME#23,destination_total#105L])
   +- HashAggregate(keys=[DEST_COUNTRY_NAME#23], functions=[sum(count#25)])
      +- Exchange hashpartitioning(DEST_COUNTRY_NAME#23, 200), ENSURE_REQUIREMENTS, [id=#230]
         +- HashAggregate(keys=[DEST_COUNTRY_NAME#23], functions=[partial_sum(count#25)])
            +- FileScan csv [DEST_COUNTRY_NAME#23,count#25] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/home/duonghung/Self-study-Spark-The-Definitive-Guide/data/flight..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string,count:int>


